In [1]:
!pip install evidently==0.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import pickle

from sklearn import datasets

from evidently.test_suite import TestSuite
from evidently.test_preset import DataStabilityTestPreset
from evidently.test_preset import BinaryClassificationTestPreset

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

# Data Drift Report

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Formation Data Science/Projet 7/df.csv')
df.head()
cols_ = list(df.columns)
cols_.remove('TARGET')
cols_.remove('NAME_FAMILY_STATUS_Unknown')
cols_.remove('NAME_INCOME_TYPE_Maternity leave')
df = df[cols_]
df_test = pd.read_csv('/content/drive/MyDrive/Formation Data Science/Projet 7/df_test.csv')
df_test.head()

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,100001,0,0,0,0,135000.0,568800.0,20560.5,450000.0,0.018850,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100005,1,0,0,0,99000.0,222768.0,17370.0,180000.0,0.035792,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100013,1,1,0,0,202500.0,663264.0,69777.0,630000.0,0.019101,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.0
3,100028,0,0,0,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0
4,100038,1,1,1,1,180000.0,625500.0,32067.0,625500.0,0.010032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data_drift_report = Report(metrics=[
    DataDriftPreset(),
])

data_drift_report.run(current_data=df_test[:10000], reference_data=df[:10000], column_mapping=None)
data_drift_report

In [ ]:
data_drift_report.save_html("/content/drive/MyDrive/Formation Data Science/Projet 7/data_drift_report.html")

In [ ]:
cols_ = ['AMT_GOODS_PRICE', 'BURO_CREDIT_TYPE_Consumer credit_MEAN', 'BURO_CREDIT_TYPE_Credit card_MEAN', 'AMT_CREDIT', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 
         'CC_AMT_DRAWINGS_CURRENT_SUM', 'BURO_CREDIT_TYPE_Mortgage_MEAN', 'BURO_CREDIT_TYPE_Car loan_MEAN', 'FLAG_DOCUMENT_3']
for col in cols_:
    if not col in df.columns:
        print(f"Problem for: {col}")
sub_df = df.loc[:,cols_]
sub_df_test = df_test.loc[:,cols_]

In [ ]:
print(sub_df.shape)
print(sub_df_test.shape)

In [ ]:
data_drift_report = Report(metrics=[
    DataDriftPreset(),
])
data_drift_report.run(current_data=sub_df_test, reference_data=sub_df, column_mapping=None)
data_drift_report

In [ ]:
data_drift_report.save_html("/content/drive/MyDrive/Formation Data Science/Projet 7/data_drift_partial_report.html")

# Model stability study

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Formation Data Science/Projet 7/df.csv')
df.head()
MODEL = pickle.load(open('/content/drive/MyDrive/Formation Data Science/Projet 7/best_model_2.pkl', 'rb'))
X = pickle.load(open('/content/drive/MyDrive/Formation Data Science/Projet 7/X.pkl', 'rb'))
X['target'] = df['TARGET']

In [ ]:
x_cols = list(X.columns)
x_cols.remove('target')
x_cols.remove('SK_ID_CURR')
print(len(x_cols))
print('target' in X.loc[:,x_cols].columns)

In [ ]:
predictions = MODEL.predict(X.loc[:,x_cols])

In [ ]:
X['prediction'] = predictions

In [ ]:
print(X['prediction'].value_counts())

In [ ]:
label_binary_classification_performance = TestSuite(tests=[
    BinaryClassificationTestPreset(),
])

label_binary_classification_performance.run(reference_data=X.iloc[:25000], current_data=X.iloc[290000:])
label_binary_classification_performance

In [ ]:
label_binary_classification_performance.save_html("/content/drive/MyDrive/Formation Data Science/Projet 7/label_binary_classification_performance.html")